# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,14.65,77,25,11.29,SH,1718059023
1,1,puerto natales,-51.7236,-72.4875,0.97,97,100,1.40,CL,1718058817
2,2,nagqu,31.5000,92.0000,3.76,76,2,2.19,CN,1718059023
3,3,port-aux-francais,-49.3500,70.2167,4.35,74,100,21.25,TF,1718058912
4,4,adamstown,-25.0660,-130.1015,22.69,81,100,7.28,PN,1718059023


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_city=city_data_df.hvplot.points("Lng","Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    size = "Humidity",
    color = "City"  
)
# YOUR CODE HERE

# Display the map
map_plot_city
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
results_city=city_data_df[(city_data_df['Humidity'] < 70) & 
                                  (city_data_df['Cloudiness'] == 0) & 
                                  (city_data_df['Max Temp'] <= 25) & 
                                  (city_data_df['Max Temp'] >=15)]
# YOUR CODE HERE

# Drop any rows with null values
results_city_null_values= results_city.dropna()
# YOUR CODE HERE

# Display sample data
results_city_null_values
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,east london,-33.0153,27.9116,16.19,51,0,3.73,ZA,1718059026
35,35,khash,28.2211,61.2158,24.43,13,0,2.00,IR,1718059030
156,156,dryden,49.7833,-92.7503,22.16,23,0,4.12,CA,1718059054
158,158,port alfred,-33.5906,26.8910,17.75,52,0,4.72,ZA,1718059055
182,182,callao,-12.0667,-77.1500,21.34,68,0,4.63,PE,1718059060
203,203,yelizovo,53.1873,158.3805,16.08,55,0,2.00,RU,1718059064
204,204,port saint john's,-31.6229,29.5448,16.12,57,0,3.20,ZA,1718059064
263,263,kapuskasing,49.4169,-82.4331,17.05,39,0,5.14,CA,1718059076
286,286,kingscliff,-28.2667,153.5667,15.98,67,0,3.09,AU,1718059081
340,340,karratha,-20.7377,116.8463,18.83,54,0,2.47,AU,1718059090


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=results_city[['City', 'Lat', 'Lng', 'Humidity']].copy()
# YOUR CODE HERE 'Country'

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''
# YOUR CODE HERE

# Display sample data
hotel_df
# YOUR CODE HERE

,City,Lat,Lng,Humidity,Hotel Name
16,east london,-33.0153,27.9116,51,
35,khash,28.2211,61.2158,13,
156,dryden,49.7833,-92.7503,23,
158,port alfred,-33.5906,26.8910,52,
182,callao,-12.0667,-77.1500,68,
203,yelizovo,53.1873,158.3805,55,
204,port saint john's,-31.6229,29.5448,57,
263,kapuskasing,49.4169,-82.4331,39,
286,kingscliff,-28.2667,153.5667,67,
340,karratha,-20.7377,116.8463,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 1000# YOUR CODE HERE
params = {"apiKey": geoapify_key,
    "limit":20,
    'categories':'accommodation.hotel'
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"# YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}"# YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)# YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json()# YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
east london - nearest hotel: No hotel found
khash - nearest hotel: No hotel found
dryden - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
callao - nearest hotel: Casa Ronald
yelizovo - nearest hotel: Николина Усадьба
port saint john's - nearest hotel: Outback Inn
kapuskasing - nearest hotel: Travelodge
kingscliff - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
chenaran - nearest hotel: No hotel found
beerwah - nearest hotel: No hotel found
taiyuan - nearest hotel: 汾酒大厦
kandahar - nearest hotel: No hotel found
port shepstone - nearest hotel: No hotel found


,City,Lat,Lng,Humidity,Hotel Name
16,east london,-33.0153,27.9116,51,No hotel found
35,khash,28.2211,61.2158,13,No hotel found
156,dryden,49.7833,-92.7503,23,No hotel found
158,port alfred,-33.5906,26.8910,52,No hotel found
182,callao,-12.0667,-77.1500,68,Casa Ronald
203,yelizovo,53.1873,158.3805,55,Николина Усадьба
204,port saint john's,-31.6229,29.5448,57,Outback Inn
263,kapuskasing,49.4169,-82.4331,39,Travelodge
286,kingscliff,-28.2667,153.5667,67,No hotel found
340,karratha,-20.7377,116.8463,54,Karratha International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display 

# Configure the map plot 
map_plot_city2 = hotel_df.hvplot.points("Lng","Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height =600,
    size = "Humidity",
    color = "City",
    hover_cols = ['City', 'Country', 'Hotel Name'] 
)
# YOUR CODE HERE

# Display the map
map_plot_city2
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)